In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import List, Optional

## 1. Structural Elements 
### Pattern: Composite Pattern - Base logic
#### Foundation of our tree structure

In [2]:
class Element(ABC):
    """Base interface for all tree nodes."""
    @abstractmethod
    def render(self) -> str:
        pass

class CompositeElement(Element):
    """Base class for containers that can hold other elements."""
    def __init__(self):
        self._children: List[Element] = []

    def add(self, element: Element):
        self._children.append(element)

    def render_children(self) -> str:
        return "".join([child.render() for child in self._children])

## 2. Abstract Factory Implementation
### Pattern: Abstract Factory
#### Purpose: Creating families of related objects (HTML vs Plain Text) without binding to specific classes.

In [3]:
# --- Abstract Products ---

class Header(Element):
    def __init__(self, text: str, level: int = 1):
        self.text = text
        self.level = level

class Paragraph(Element):
    def __init__(self, text: str):
        self.text = text

class Container(CompositeElement):
    """Generic container (e.g., div or section)."""
    pass

# --- Concrete Products (HTML Family) ---

class HTMLHeader(Header):
    def render(self) -> str:
        return f"<h{self.level}>{self.text}</h{self.level}>\n"

class HTMLParagraph(Paragraph):
    def render(self) -> str:
        return f"<p>{self.text}</p>\n"

class HTMLContainer(Container):
    def render(self) -> str:
        return f"<div class='section'>\n{self.render_children()}</div>\n"

# --- Concrete Products (Plain Text Family) ---

class TextHeader(Header):
    def render(self) -> str:
        underline = "=" * len(self.text) if self.level == 1 else "-" * len(self.text)
        return f"{self.text}\n{underline}\n"

class TextParagraph(Paragraph):
    def render(self) -> str:
        return f"{self.text}\n"

class TextContainer(Container):
    def render(self) -> str:
        return f"\n--- Section ---\n{self.render_children()}\n"

# --- The Factory Interface ---

class DocumentFactory(ABC):
    @abstractmethod
    def create_header(self, text: str, level: int = 1) -> Header:
        pass

    @abstractmethod
    def create_paragraph(self, text: str) -> Paragraph:
        pass

    @abstractmethod
    def create_container(self) -> Container:
        pass

# --- Concrete Factories ---

class HTMLFactory(DocumentFactory):
    def create_header(self, text: str, level: int = 1) -> Header:
        return HTMLHeader(text, level)

    def create_paragraph(self, text: str) -> Paragraph:
        return HTMLParagraph(text)

    def create_container(self) -> Container:
        return HTMLContainer()

class TextFactory(DocumentFactory):
    def create_header(self, text: str, level: int = 1) -> Header:
        return TextHeader(text, level)

    def create_paragraph(self, text: str) -> Paragraph:
        return TextParagraph(text)

    def create_container(self) -> Container:
        return TextContainer()

## 3. Builder Implementation
### Pattern: Builder
#### Purpose: Simplify the creation of complex nested structures.
#### The client does not need to manually instantiate and call .add().

In [4]:
class DocumentBuilder:
    def __init__(self, factory: DocumentFactory):
        self.factory = factory
        # The container stack allows us to create nested structures easily
        self._root = self.factory.create_container()
        self._stack = [self._root]

    def add_title(self, text: str) -> DocumentBuilder:
        header = self.factory.create_header(text, level=1)
        self._current_container().add(header)
        return self

    def add_subtitle(self, text: str) -> DocumentBuilder:
        header = self.factory.create_header(text, level=2)
        self._current_container().add(header)
        return self

    def add_text(self, text: str) -> DocumentBuilder:
        para = self.factory.create_paragraph(text)
        self._current_container().add(para)
        return self

    def start_section(self) -> DocumentBuilder:
        new_container = self.factory.create_container()
        self._current_container().add(new_container)
        self._stack.append(new_container)
        return self

    def end_section(self) -> DocumentBuilder:
        if len(self._stack) > 1:
            self._stack.pop()
        return self

    def get_result(self) -> Element:
        return self._root

    def _current_container(self) -> CompositeElement:
        return self._stack[-1]

## 4. Singleton Implementation
### Pattern: Singleton
#### Purpose: Manage global document state. Ensures we don't edit two
#### documents simultaneously within one application context.

In [5]:
class DocumentManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(DocumentManager, cls).__new__(cls)
            cls._instance.current_document = None
            print("[System] DocumentManager Initialized (Singleton)")
        return cls._instance

    def set_document(self, doc: Element):
        self.current_document = doc

    def print_document(self):
        if self.current_document:
            print(self.current_document.render())
        else:
            print("[Error] No document loaded.")

## 5. Client Code / Testing

In [6]:
def client_code(factory: DocumentFactory, format_name: str):
    print(f"\n>>> Creating document in {format_name} format...\n")
    
    # Use Builder to create the structure
    # Note: The construction logic is identical for both formats!
    builder = DocumentBuilder(factory)
    
    doc_tree = (builder
        .add_title(f"My {format_name} Report")
        .add_text("This represents the introduction of the report.")
        .start_section() # Nested Level 1
            .add_subtitle("Chapter 1: Patterns")
            .add_text("We are using Abstract Factory and Builder.")
            .start_section() # Nested Level 2
                .add_text("Deeply nested note inside Chapter 1.")
            .end_section()
        .end_section()
        .add_text("Conclusion text.")
        .get_result()
    )

    # Use Singleton to manage the document
    manager = DocumentManager()
    manager.set_document(doc_tree)
    manager.print_document()

if __name__ == "__main__":
    # Test 1: Create HTML Document
    html_factory = HTMLFactory()
    client_code(html_factory, "HTML")

    print("-" * 40)

    # Test 2: Create Plain Text Document
    # Client code doesn't change, only the factory passed changes
    text_factory = TextFactory()
    client_code(text_factory, "Plain Text")

    print("-" * 40)
    
    # Verify Singleton behavior
    m1 = DocumentManager()
    m2 = DocumentManager()
    print(f"\nSingleton Check: m1 is m2? {m1 is m2}")


>>> Creating document in HTML format...

[System] DocumentManager Initialized (Singleton)
<div class='section'>
<h1>My HTML Report</h1>
<p>This represents the introduction of the report.</p>
<div class='section'>
<h2>Chapter 1: Patterns</h2>
<p>We are using Abstract Factory and Builder.</p>
<div class='section'>
<p>Deeply nested note inside Chapter 1.</p>
</div>
</div>
<p>Conclusion text.</p>
</div>

----------------------------------------

>>> Creating document in Plain Text format...


--- Section ---
My Plain Text Report
This represents the introduction of the report.

--- Section ---
Chapter 1: Patterns
-------------------
We are using Abstract Factory and Builder.

--- Section ---
Deeply nested note inside Chapter 1.


Conclusion text.


----------------------------------------

Singleton Check: m1 is m2? True
